In [1]:
import Data.Manifold
import Data.Manifold.Web
import Linear.V2
import Data.Foldable (toList)
import Control.Arrow

From [dynamic-plot](http://hackage.haskell.org/packages/dynamic-plot):

In [2]:
import Graphics.Dynamic.Plot.R2

In [3]:
import Data.Colour
import Data.Colour.Names

In [18]:
iWeb :: PointsWeb ℝ² (ℝ²)
iWeb = fromWebNodes euclideanMetric $ (fst&&&fst)<$>
   [ (V2 0 0, blue), (V2 1 0, beige), (V2 2 0, teal), (V2 3 0, magenta)
   , (V2 0 1, red),  (V2 1 1, violet),(V2 2 1, green),(V2 3 1, turquoise)
   , (V2 0 2, gray), (V2 1 2, indigo),(V2 2 2, cyan), (V2 3 2, lavender)
   , (V2 0 3, gray), (V2 1 3, indigo),(V2 2 3, cyan), (V2 3 3, royalblue) ]
-- plotWindow [plot iWeb]

In [19]:
import Data.Semigroup (getOption)
mapM_ print $ second (map $ second getOption) <$> sampleEntireWeb_2Dcartesian_lin (coerceWebDomain iWeb) 4 4
map (fst . fst) . toList $ localFocusWeb iWeb

(0.375,[(0.375,Just (V2 0.84375 0.375)),(1.125,Just (V2 1.78125 0.375)),(1.875,Just (V2 1.96875 0.375)),(2.625,Just (V2 2.90625 0.375))])
(1.125,[(0.375,Just (V2 0.84375 1.125)),(1.125,Just (V2 1.78125 1.125)),(1.875,Just (V2 1.96875 1.125)),(2.625,Just (V2 2.90625 1.125))])
(1.875,[(0.375,Just (V2 0.84375 1.875)),(1.125,Just (V2 1.78125 1.875)),(1.875,Just (V2 1.96875 1.875)),(2.625,Just (V2 2.90625 1.875))])
(2.625,[(0.375,Just (V2 0.84375 2.625)),(1.125,Just (V2 1.78125 2.625)),(1.875,Just (V2 1.96875 2.625)),(2.625,Just (V2 2.90625 2.625))])

[V2 3.0 2.0,V2 3.0 3.0,V2 0.0 2.0,V2 0.0 3.0,V2 1.0 3.0,V2 2.0 3.0,V2 1.0 2.0,V2 2.0 2.0,V2 3.0 1.0,V2 3.0 0.0,V2 0.0 0.0,V2 0.0 1.0,V2 1.0 1.0,V2 2.0 1.0,V2 1.0 0.0,V2 2.0 0.0]

In [6]:
testPlane :: Cutplane ℝ²
testPlane = Cutplane (V2 1.5 0) (Stiefel1 $ V2 1 0)
fst <$> sliceWeb_lin iWeb testPlane

[V2 1.5 3.0,V2 1.5 2.0,V2 1.5 1.0,V2 1.5 0.0]

In [7]:
length $ (fst***fst) <$> webEdges iWeb

24

In [8]:
testPlane`cutPosBetween`(V2 0 0, V2 2 0)

Option {getOption = Just (D¹ {xParamD¹ = 0.5})}

In [9]:
map (fst***fst) . webEdges . fromWebNodes euclideanMetric $ sliceWeb_lin iWeb testPlane

[(V2 1.5 3.0,V2 1.5 2.0),(V2 1.5 2.0,V2 1.5 1.0),(V2 1.5 1.0,V2 1.5 0.0)]

In [10]:
splitToGridLines (coerceWebDomain iWeb) $ cartesianGrid2D ((0,3),4) ((0,3),4)

[(((0.375,0.375),GridPlanes {_gridPlaneNormal = (1.0,0.0), _gridPlaneSpacing = (0.75,0.0), _gridPlanesCount = 4}),[((3.0,0.375),Data.Colour.SRGB.Linear.rgb 0.6442260468902662 0.2795265785776453 0.8615339261298051),((0.0,0.375),Data.Colour.SRGB.Linear.rgb 0.375 0.0 0.625),((1.0,0.375),Data.Colour.SRGB.Linear.rgb 0.8913088854174747 0.6543971413646902 0.7679306658631382),((2.0,0.375),Data.Colour.SRGB.Linear.rgb 0.0 0.21586050011389926 0.13491281257118704)]),(((0.375,1.125),GridPlanes {_gridPlaneNormal = (1.0,0.0), _gridPlaneSpacing = (0.75,0.0), _gridPlanesCount = 4}),[((3.0,1.125),Data.Colour.SRGB.Linear.rgb 0.1437730186188666 0.7511409258894178 0.6714091634590392),((0.0,1.125),Data.Colour.SRGB.Linear.rgb 0.9019825625142374 2.6982562514237408e-2 2.6982562514237408e-2),((1.0,1.125),Data.Colour.SRGB.Linear.rgb 0.756913544070779 0.19532446265220743 0.7760220267733057),((2.0,1.125),Data.Colour.SRGB.Linear.rgb 0.0 0.31387793759966187 0.125)]),(((0.375,1.875),GridPlanes {_gridPlaneNormal = (1.

In [11]:
[( ((0.0,0.0),GridPlanes {_gridPlaneNormal = (1.0,0.0), _gridPlaneSpacing = (1.0,0.0), _gridPlanesCount = 4})
 , [] )
,( ((0.0,1.0),GridPlanes {_gridPlaneNormal = (1.0,0.0), _gridPlaneSpacing = (1.0,0.0), _gridPlanesCount = 4})
 , [ ((3.0,1.0),Data.Colour.SRGB.Linear.rgb 5.126945837404324e-2 0.7454042095403874 0.6307571363461468)
   , ((0.0,1.0),Data.Colour.SRGB.Linear.rgb 1.0 0.0 0.0)
   , ((1.0,1.0),Data.Colour.SRGB.Linear.rgb 0.8549926081242338 0.2232279573168085 0.8549926081242338)
   , ((2.0,1.0),Data.Colour.SRGB.Linear.rgb 0.0 0.21586050011389926 0.0) ] )
,( ((0.0,2.0),GridPlanes {_gridPlaneNormal = (1.0,0.0), _gridPlaneSpacing = (1.0,0.0), _gridPlanesCount = 4})
 , [((3.0,2.0),Data.Colour.SRGB.Linear.rgb 0.7912979403326302 0.7912979403326302 0.9559733532492861)
 , ((0.0,2.0),Data.Colour.SRGB.Linear.rgb 0.21586050011389926 0.21586050011389926 0.21586050011389926),((1.0,2.0),Data.Colour.SRGB.Linear.rgb 7.036009569659588e-2 0.0 0.2232279573168085),((2.0,2.0),Data.Colour.SRGB.Linear.rgb 0.0 1.0 1.0)]),(((0.0,3.0),GridPlanes {_gridPlaneNormal = (1.0,0.0), _gridPlaneSpacing = (1.0,0.0), _gridPlanesCount = 4}),[((3.0,3.0),Data.Colour.SRGB.Linear.rgb 5.2860647023180246e-2 0.14126329114027164 0.7529422167760779),((0.0,3.0),Data.Colour.SRGB.Linear.rgb 0.21586050011389926 0.21586050011389926 0.21586050011389926),((1.0,3.0),Data.Colour.SRGB.Linear.rgb 7.036009569659588e-2 0.0 0.2232279573168085),((2.0,3.0),Data.Colour.SRGB.Linear.rgb 0.0 1.0 1.0)])]

Parse error (line 2, column 8): parse error (possibly incorrect indentation or mismatched brackets)

Parse error (line 3, column 1): parse error on input ‘,’

Parse error (line 8, column 1): parse error on input ‘,’